# YOLO Pipeline - YOLOv8 для детекции и классификации

Пайплайны для:
- Object Detection
- Instance Segmentation  
- Image Classification
- Pose Estimation

In [ ]:
!pip install ultralytics opencv-python pandas -q

In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

print("✓ Библиотеки загружены!")

## 1. Object Detection

In [ ]:
# === НАСТРОЙКИ ===
DATA_YAML = 'data.yaml'  # Конфиг с путями к данным
# Пример data.yaml:
# path: ./dataset
# train: images/train
# val: images/val
# nc: 80  # количество классов
# names: ['person', 'car', ...]  # названия классов

MODEL_SIZE = 'yolov8n.pt'  # n, s, m, l, x (от маленькой к большой)
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 16

In [ ]:
# Загрузка модели
model = YOLO(MODEL_SIZE)

# Обучение
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=0,  # GPU, или 'cpu'
    project='yolo_detection',
    name='exp',
    # Дополнительные параметры:
    # patience=50,  # Early stopping
    # save=True,
    # cache=True,  # Кэширование изображений
)

print("✓ Обучение завершено!")

In [ ]:
# Валидация
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

In [ ]:
# Предсказания
TEST_DIR = './test_images'

results = model.predict(
    source=TEST_DIR,
    save=True,
    save_txt=True,  # Сохранение в формате YOLO
    conf=0.25,  # Порог уверенности
    iou=0.45,   # NMS threshold
)

# Обработка результатов
predictions = []
for r in results:
    boxes = r.boxes
    for box in boxes:
        predictions.append({
            'image': r.path,
            'class': int(box.cls),
            'confidence': float(box.conf),
            'bbox': box.xyxy.cpu().numpy().tolist()
        })

pd.DataFrame(predictions).to_csv('yolo_detections.csv', index=False)
print("✓ Предсказания сохранены!")

## 2. Instance Segmentation

In [ ]:
# Загрузка модели сегментации
seg_model = YOLO('yolov8n-seg.pt')

# Обучение
seg_results = seg_model.train(
    data=DATA_YAML,  # Должен содержать пути к маскам
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
)

# Предсказания
seg_predictions = seg_model.predict(
    source=TEST_DIR,
    save=True,
)

print("✓ Сегментация завершена!")

## 3. Image Classification

In [ ]:
# Модель классификации
cls_model = YOLO('yolov8n-cls.pt')

# Обучение
# Структура данных:
# dataset/
#   train/
#     class1/
#       img1.jpg
#     class2/
#   val/
#     class1/
#     class2/

cls_results = cls_model.train(
    data='./dataset',  # Путь к папке с данными
    epochs=50,
    imgsz=224,
    batch=32,
)

# Предсказания
cls_predictions = cls_model.predict(TEST_DIR)

# Извлечение классов
class_results = []
for r in cls_predictions:
    probs = r.probs
    class_results.append({
        'image': r.path,
        'predicted_class': int(probs.top1),
        'confidence': float(probs.top1conf)
    })

pd.DataFrame(class_results).to_csv('yolo_classification.csv', index=False)

## 4. Экспорт модели

In [ ]:
# Экспорт в разные форматы
# model.export(format='onnx')     # ONNX
# model.export(format='torchscript')  # TorchScript
# model.export(format='coreml')   # CoreML для iOS
# model.export(format='tflite')   # TensorFlow Lite

## 5. Inference на видео

In [ ]:
# Обработка видео
# results = model.predict(
#     source='video.mp4',
#     save=True,
#     stream=True,  # Для потокового видео
# )

# for r in results:
#     # Обработка каждого кадра
#     pass